<a id="top"></a>
# MAST API Tutorial - Astroquery API

## Introduction and Goals:
This is a beginner tutorial on accessing the MAST database using the Astroquery API. By the end of this tutorial, you will:

* Understand how to search for observations hosted on the MAST archive
* Download data products corresponding to your observations of interest
* Create a visual display of the downloaded data

## Table of Contents

* [Imports](#Imports)


* [Three ways to search for MAST observations](#Three-Ways-to-Search-for-MAST-Observations)
    - [By Region](#1.-By-Region)
    - [By Object Name](#2.-By-Object-Name)
    - [By Criteria](#crit)
    
    
* [Getting Associated Data Products](#Getting-Associated-Data-Products)
    - [Performing a Product Query](#Performing-a-Product-Query)
    - [Filtering Data Products](#Filtering-the-Data-Products)
    
    
* [Downloading Products](#Downloading-Products)


* [Displaying Data](#Displaying-Data)


* [Further Reading](#Further-Reading)

## Imports

The only import we need for the MAST API is `astroquery.mast`. 

`astropy` and `matplotlib` are for opening and plotting the data, respectively.

In [ ]:
from astroquery.mast import Observations

from astropy.io import fits

import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
%matplotlib inline 

## Three Ways to Search for MAST Observations

### 1. By Region
You can use `query_region` to find objects based on coordinates and radius.  
The coordinates can be given as a string or `astropy.coordinates` object, and the radius as a string or float object. If no radius is specified, the default is 0.2 degrees.

In [ ]:
# This will give a warning that the coordinates are being interpreted as an ICRS coordinate provided in degrees
obsByRegion = Observations.query_region("322.49324 12.16683")

In [ ]:
# How many results? Let's look at the first ten.
print("Number of results:",len(obsByRegion))
print(obsByRegion[:10])

### 2. By Object Name

The `query.object` method allows you to search based on object name and optional radius argument.  
The object name is first resolved to sky coordinates, and then the resolved region is queried as above.

In [ ]:
obsByName = Observations.query_object("M51",radius=".005 deg")

In [ ]:
print("Number of results:",len(obsByName))
print(obsByName[:10])

### 3. By Other Criteria (with or without name/region) <a id="crit"></a>

To search for observations based on additonal parameters, you can use `query_criteria`. In a sense, this is a more powerful version of the tools above, as you can still search by coordinates and objectname; however, you can inculde additional desired criteria. You may also run the search without specifying a name or coordinates.

To perform the search, give your critera as keyword arguments. Valid criteria are listed [here](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html). Some examples are: "coordinates", "t_exptime" (exposure time), and "s_dec" (source declination).

In [ ]:
obsByCriteria = Observations.query_criteria(obs_collection=["HLA"], s_dec=[50,60], 
                                            calib_level=[3], proposal_pi="Mould*", 
                                            dataproduct_type="IMAGE", t_max=[49800,49820])

#obsByCriteria = Observations.query_criteria(obs_id = 'hst_05766_04_wfpc2_total_wf')

In [ ]:
obsByCriteria.show_in_notebook(display_length=5)

## Getting Associated Data Products

### Performing a Product Query


Each observation returned from a MAST query can have one or more associated data products. When you input observations or observation ids (“obs_id”), `get_product_list` will return a table containing the associated data products. The product fields are documented [here](https://mast.stsci.edu/api/v0/_productsfields.html).

Since we already have a list of observations, we can use that as the starting point for our query. To keep it simple, let's look at only the last observation from our search above.

In [ ]:
# Let's select a small subset from our critera search above
newObsList = obsByCriteria[3:]

# Now we get the list of products associated with that observation
dataProducts = Observations.get_product_list(newObsList)

In [ ]:
dataProducts.show_in_notebook(display_length=5)

### Filtering the Data Products

After the data products have been retrieved, you can use `filter_products` to download only data products that meet your given criteria. Available filters are listed [here](https://mast.stsci.edu/api/v0/_productsfields.html). Some examples are: “mrp_only” (Minimum Recommended Products) and “extension” (file extension).

Note: the ‘AND’ operation is performed for a list of filters, and the ‘OR’ operation is performed within a filter set.

In [ ]:
scienceProducts = Observations.filter_products(dataProducts, productType=["SCIENCE"],
                                               mrp_only=False)

In [ ]:
scienceProducts.show_in_notebook(display_length=5)

## Downloading Products


In additon to pre-filtering, `download_products` also allows you to filter data as you request the download. In this instance, we will only download the drizzled files (drz.fits).

Products will by default be downloaded into the current working directory, in a subdirectory called "mastDownload."<br>
The full local file paths will have the form "mastDownload/Mission/Observation ID/file."

Note: `download_products` includes caching by default, so if the files have been downloaded, they will not be downloaded again unless caching is turned off.

In [ ]:
manifest = Observations.download_products(scienceProducts, mrp_only=False,
                                          extension=("drz.fits"), cache=True)

In [ ]:
manifest.show_in_notebook()

## Displaying Data

At this point the data is ready for analysis, and we are done using `mast.astroquery`. <br>
Below we take a look at the data files using `astropy` and `matplotlib`.

In [ ]:
file1 = fits.open(manifest['Local Path'][0])
file2 = fits.open(manifest['Local Path'][1])

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2)
f.set_figheight(5)
f.set_figwidth(12)
ax1.imshow(file1[0].data, cmap="inferno", norm=SymLogNorm(linthresh=0.03,vmin=0, vmax=1.5))
ax2.imshow(file2['SCI'].data, cmap="inferno", norm=SymLogNorm(linthresh=0.03,vmin=-0.01, vmax=1.5))

## Further Reading
Full documentation on **`astroquery.mast`** can be found [here](https://astroquery.readthedocs.io/en/latest/mast/mast.html).

## About this Notebook
For additonal questions, comments, or feedback, please email `archive@stsci.edu`. 

**Authors:** Thomas Dutkiewicz, Scott Fleming <br>
**Keywords:** MAST, astroquery <br>
**Created On:** 2022-06-28 <br>
**Next Review:** 2023-06

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 